In [1]:
import pandas as pd



#Reload the dataset
data = pd.read_excel('../atlanta_east/Daily_Data_AtlantaEastNA.xlsx')

# Convert the 'Date' column to datetime
data['Date'] = pd.to_datetime(data['Date'])

data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day
data['DayOfWeek'] = data['Date'].dt.dayofweek  # Monday=0, Sunday=6

# Dropping the original 'Date' column
data = data.drop(columns=['DATE OUT', 'Date', 'Unnamed: 0'])

# Checking the updated dataframe
data.head()


,mean_rate_day,max_rate_day,min_rate_day,mean_rate_mile,max_rate_mile,min_rate_mile,2015,2016,2017,2018,...,2022,SumOfFLEET All,Utilization Rate All,temp,precip,snowdepth,Year,Month,Day,DayOfWeek
0,50.835000,69.99,31.68,0.435600,0.8712,0.0000,NaN,NaN,NaN,NaN,...,NaN,75,0.613333,42.7,0.060,0.0,2015,1,1,3
1,48.480000,69.99,32.46,0.620400,0.9900,0.0000,NaN,NaN,NaN,NaN,...,NaN,73,0.671233,45.7,0.851,0.0,2015,1,2,4
2,42.082857,69.99,33.44,0.818743,0.9900,0.0000,NaN,NaN,NaN,NaN,...,NaN,70,0.642857,51.2,0.221,0.0,2015,1,3,5
3,47.390000,69.99,35.19,0.913733,0.9900,0.8712,NaN,NaN,NaN,NaN,...,NaN,72,0.597222,57.7,1.319,0.0,2015,1,4,6
4,29.990000,29.99,29.99,0.250000,0.2500,0.2500,NaN,NaN,NaN,NaN,...,NaN,72,0.569444,40.8,0.000,0.0,2015,1,5,0


In [2]:
# Data Processing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
missing_values = data.isnull().sum()

In [4]:
#%% train and test sets
data=data.dropna()
split = int(.8*len(data))

X_train = data.loc[:split,:].drop(['Utilization Rate All'],axis=1)
y_train = data.loc[:split,'Utilization Rate All']

X_test = data.loc[split:,:].drop(['Utilization Rate All'],axis=1)
y_test = data.loc[split:,'Utilization Rate All']


In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Initialize the Linear Regression model
lr = LinearRegression()

# Train the model
lr.fit(X_train, y_train)

# Predict on the test set
y_pred_lr = lr.predict(X_test)

# Calculate the mean squared error for the Linear Regression model
mse_lr = mean_squared_error(y_test, y_pred_lr)
print(f"Linear Regression MSE: {mse_lr}")

r_squared = r2_score(y_test, y_pred_lr)
print("R-squared:", r_squared)

# Sorting the coefficients in descending order
sorted_coeffs = sorted(zip(X_train.columns, lr.coef_), key=lambda x: x[1], reverse=True)

# Print the intercept
print("Intercept:", lr.intercept_)

# Print the sorted coefficients for each feature
for feature, coef in sorted_coeffs:
    print(f"{feature}: {coef}")


print('________________________________________________')

ValueError: Found array with 0 sample(s) (shape=(0, 22)) while a minimum of 1 is required by LinearRegression.

In [ ]:
# Linear Regression Predictions vs Actual
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.scatter(y_test, y_pred_lr, color='red', alpha=0.5, label='Predictions')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='blue', label='y=x line')
plt.title('Linear Regression: Predictions vs Actual')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.legend()
plt.axis('square')

In [ ]:
features = data.drop(['Utilization Rate All'], axis=1)
labels = data['Utilization Rate All']

# Split the data into training and test sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.20, random_state = 12)

print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

feature_list = list(features.columns)

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 12)
# Train the model on training data
rf.fit(train_features, train_labels);

# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
mse_rf = mean_squared_error(test_labels, predictions)
print(f"RandomForest MSE: {mse_rf}")
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:
from sklearn.metrics import r2_score
score = r2_score(test_labels, predictions)
print("The accuracy of our model is {}%".format(round(score, 2) *100))

In [ ]:
import matplotlib.pyplot as plt

#Random Forest Predictions vs Actual
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.scatter(test_labels, predictions, color='red', alpha=0.5, label='Predictions')
plt.plot([min(test_labels), max(test_labels)], [min(test_labels), max(test_labels)], color='blue', label='y=x line')
plt.title('RandomForest: Predictions vs Actual')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.legend()
plt.axis('square')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
x = data.drop(['Utilization Rate All', 'Year'], axis=1)
y = data['Utilization Rate All']

xtrain, xtest, ytrain, ytest=train_test_split(x, y, random_state=12,
             test_size=0.15)
# with new parameters
gbr = GradientBoostingRegressor(n_estimators=600,
    max_depth=5,
    learning_rate=0.01,
    min_samples_split=3)
# with default parameters
#gbr = GradientBoostingRegressor()

gbr.fit(xtrain, ytrain)

ypred = gbr.predict(xtest)


x_ax = range(len(ytest))
plt.scatter(x_ax, ytest, s=5, color="blue", label="original")
plt.plot(x_ax, ypred, lw=0.8, color="red", label="predicted")
plt.legend()
plt.show()

In [ ]:
#Gradient Boosting Predictions vs Actual
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.scatter(ytest, ypred, color='red', alpha=0.5, label='Predictions')
plt.plot([min(ytest), max(ytest)], [min(ytest), max(ytest)], color='blue', label='y=x line')
plt.title('Gradient Boost: Predictions vs Actual')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.legend()
plt.axis('square')

plt.tight_layout()
plt.show()

# Calculate the absolute errors
errors = abs(ypred - ytest)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
print(ypred.shape[0])
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / ytest)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:
mse = mean_squared_error(ytest,ypred)
print(mse)

In [ ]:
from sklearn.metrics import r2_score
score = r2_score(ytest, ypred)
print("The accuracy of our model is {}%".format(round(score, 2) *100))

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
x = data.drop(['Utilization Rate All', 'Year'], axis=1)
y = data['Utilization Rate All']

xtrain, xtest, ytrain, ytest=train_test_split(x, y, random_state=0,
             test_size=0.2)
# with new parameters
hgbr = HistGradientBoostingRegressor(n_estimators=300,
    max_depth=5,
    learning_rate=0.01,
    min_samples_split=3)
# with default parameters
#gbr = GradientBoostingRegressor()

hgbr.fit(xtrain, ytrain)

ypred = hgbr.predict(xtest)


x_ax = range(len(ytest))
plt.scatter(x_ax, ytest, s=5, color="blue", label="original")
plt.plot(x_ax, ypred, lw=0.8, color="red", label="predicted")
plt.legend()
plt.show()

In [ ]:
#Gradient Boosting Predictions vs Actual
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.scatter(ytest, ypred, color='red', alpha=0.5, label='Predictions')
plt.plot([min(ytest), max(ytest)], [min(ytest), max(ytest)], color='blue', label='y=x line')
plt.title('HistGradient Boost: Predictions vs Actual')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.legend()
plt.axis('square')

plt.tight_layout()
plt.show()

# Calculate the absolute errors
errors = abs(ypred - ytest)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
print(ypred.shape[0])
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / ytest)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:
mse = mean_squared_error(ytest,ypred)
print(mse)
from sklearn.metrics import r2_score
score = r2_score(ytest, ypred)
print("The accuracy of our model is {}%".format(round(score, 2) *100))

In [7]:
from scipy.stats import loguniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import HistGradientBoostingRegressor
x = data.drop(['Utilization Rate All', 'Year'], axis=1)
y = data['Utilization Rate All']
xtrain, xtest, ytrain, ytest=train_test_split(x, y, random_state=0,
             test_size=0.2)

param_distributions = {
    "max_iter": [3, 10, 30, 100, 300, 1000],
    "max_leaf_nodes": [2, 5, 10, 20, 50, 100],
    "learning_rate": loguniform(0.01, 1),
}
search_cv = RandomizedSearchCV(
    HistGradientBoostingRegressor(),
    param_distributions=param_distributions,
    scoring="neg_mean_absolute_error",
    n_iter=20,
    random_state=0,
    n_jobs=2,
)
search_cv.fit(xtrain, ytrain)

columns = [f"param_{name}" for name in param_distributions.keys()]
columns += ["mean_test_error", "std_test_error"]
cv_results = pd.DataFrame(search_cv.cv_results_)
cv_results["mean_test_error"] = -cv_results["mean_test_score"]
cv_results["std_test_error"] = cv_results["std_test_score"]
cv_results[columns].sort_values(by="mean_test_error")

,param_max_iter,param_max_leaf_nodes,param_learning_rate,mean_test_error,std_test_error
16,300,5,0.05929,0.022667,0.003818
1,100,5,0.160519,0.023332,0.004235
6,300,20,0.047293,0.023431,0.003973
12,100,20,0.067503,0.023473,0.004274
14,300,100,0.01864,0.023707,0.004139
9,100,20,0.083745,0.023943,0.004213
2,30,50,0.176656,0.024065,0.003806
19,100,10,0.215543,0.024229,0.003700
0,1000,2,0.125207,0.024447,0.002631
7,1000,2,0.054511,0.024464,0.003297


In [9]:
error = -search_cv.score(xtest, ytest)
print(f"On average, our HGBT regressor makes an error of {error:.2f} k$")

On average, our HGBT regressor makes an error of 0.03 k$
